## <b>M5_종합실습</b>

### 1.정형 데이터 분석

- csv 데이터 불러오기 : 환경부 국가미세먼지정보센터_대기배출원조사 결과_20201231.csv

In [ ]:
import pandas as pd

df = pd.read_csv('data_set/환경부 국가미세먼지정보센터_대기배출원조사 결과_20201231.csv', encoding='euc-kr')
df

In [ ]:
df.info()

- 컬럼 추가하고 계산하기

In [ ]:
df['평균'] = df.mean(axis=1, numeric_only=True) #axis=1 : 행단위로 평균을 구하라
df['합계'] = df.sum(axis=1, numeric_only=True)

In [ ]:
df

In [ ]:
df['사업장평균'] = df.iloc[:,2:5].mean(axis=1, numeric_only=True)

In [ ]:
df

- 데이터 프레임을 이미지로 내보내기

In [ ]:
import dataframe_image as dfi
dfi.export(df, 'mean_sum.png', max_rows=10, max_cols=-1)

### 2.웹 데이터 추출

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

- 대한민국헌법(1장~10장, 부칙) 텍스트 데이터 가져오기 

In [ ]:
url = 'https://www.law.go.kr/LSW/lsInfoP.do?efYd=19880225&lsiSeq=61603#0000'

driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(10)

In [ ]:
law_text = driver.find_element(By.XPATH, '//*[@id="conScroll"]').text
driver.quit()

### 3.텍스트 분석(빈도 분석)

- 형태소 분석

In [ ]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

kiwi = Kiwi()
stopwords = Stopwords()

tokens = kiwi.tokenize(law_text, stopwords=stopwords)
print(tokens[1000:1001])

- 단어만 추출

In [ ]:
token_text = []

for token in tokens:
    token_text.append(token.form)

print(token_text[:100])

- 단어 빈도수 측정

In [ ]:
from collections import Counter

In [ ]:
c = Counter(token_text)
c

In [ ]:
top_c = c.most_common(10)

In [ ]:
top_c = dict(top_c)

In [ ]:
top_c

- 단어 빈도 시각화(그래프)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm

In [ ]:
plt.style.use('bmh')

- 폰트 매니저에서 관리하는 폰트 확인

- 폰트 설정

In [ ]:
plt.rc('font', family='Gulim', size=7)

- 세로 막대 그래프 그리기

In [ ]:
plt.figure(figsize=(7, 5))
plt.bar(top_c.keys(), top_c.values())
plt.title('법률용어 빈도수')
plt.savefig('law_top10.png')
plt.show();

### 4.EP 메일 보내기

- EP 로그인 정보 입력(ID, PASS)

In [ ]:
import pyautogui as pg
from selenium.webdriver.common.keys import Keys
import clipboard as cb

In [ ]:
my_id = 'yooleenes'
my_pw = pg.password(text='비밀번호를 입력하세요')

- EP 접속

In [ ]:
url = 'http://swpsso.posco.net/idms/U61/jsp/login/login.jsp'

driver = webdriver.Edge()
driver.get(url)
driver.implicitly_wait(10)

In [ ]:
driver.find_element(By.XPATH, '//*[@id="username"]').send_keys(my_id)
driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(my_pw)
driver.find_element(By.XPATH, '//*[@id="loginsubmit"]').click()

pg.sleep(5)
try:
    alert = driver.switch_to.alert
    alert.accept()
except: print('계속 진행합니다.')
pg.sleep(15)

- 메일 화면 열기

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
driver.find_element(By.XPATH, '//*[@id="533982"]/a[1]/span[1]/img').click()
pg.sleep(3)

- 메일 쓰기 화면 열기

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
driver.find_element(By.XPATH, '//*[@id="Lnb"]/div[1]/a').click()
pg.sleep(3)

- 메일 정보, 본문 내용 입력

In [ ]:
mail_add = 'yooleenes@poscohrd.com'
mail_sub = '테스트 메일입니다.'
mail_con1 = '안녕하세요.ㅇㅇㅇㅇㅇㅇ'
mail_con2 = '환경부 대기정보 표입니다'
mail_con3 = '헌법 키워드 분석 그리프입니다.'

- 메일 주소 입력

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
driver.find_element(By.XPATH, '//*[@id="token-input-send_to"]').send_keys(mail_add)
driver.find_element(By.XPATH, '//*[@id="token-input-send_to"]').send_keys(Keys.ENTER)
pg.sleep(3)

- 메일 제목 입력

In [ ]:
driver.find_element(By.XPATH, '//*[@id="write_send_info"]/table[2]/tbody/tr[1]/td/input').send_keys(mail_sub)

- 메일 본문 입력(기본 문구)

In [ ]:
driver.switch_to.frame('dext_frame_WEC')
driver.switch_to.frame('dext5_design_WEC')
driver.find_element(By.XPATH,'//*[@id="dext_body"]/p[1]').send_keys(Keys.ENTER)
pg.sleep(1)
driver.find_element(By.XPATH,'//*[@id="dext_body"]/p[1]/br').send_keys(mail_con1)
driver.find_element(By.XPATH,'//*[@id="dext_body"]/p[1]/br').send_keys(Keys.ENTER)
driver.find_element(By.XPATH,'//*[@id="dext_body"]/p[1]/br').send_keys(Keys.ENTER)

- 메일 본문 입력(정형 데이터)

In [ ]:
driver.find_element(By.XPATH,'//*[@id="dext_body"]/p[1]/br').send_keys(mail_con2)

- 이미지 복사(정형 데이터 테이블)

In [ ]:
pg.hotkey('win','r')
cb.copy('D:\작업방\Level2_office\mean_sum.png')
pg.sleep(0.5)
pg.hotkey('ctrl','v')
pg.sleep(0.5)
pg.press('enter')
pg.sleep(15)

win = pg.getWindowsWithTitle('사진')[0]
win.activate()

pg.hotkey('ctrl','c')

- 이미지 붙여넣기(정형 데이터 테이블)

In [ ]:
win = pg.getWindowsWithTitle('메일쓰기')[0]
win.activate()

pg.press('enter', presses=2)
pg.hotkey('ctrl','v')
pg.sleep(1)
pg.press('tab', presses=5, interval=0.2)
pg.write('1200') #이미지 크기 설정
pg.press('tab', presses=5, interval=0.2)
pg.press('enter')

pg.press('enter')

- 메일 본문 입력(비정형 데이터)

In [ ]:
driver.find_element(By.XPATH,'//*[@id="dext_body"]/p[6]').send_keys(mail_con3)



- 이미지 복사(비정형 데이터 그래프)

In [ ]:
pg.hotkey('win','r')
cb.copy('D:\작업방\Level2_office\law_top10.png')
pg.sleep(0.5)
pg.hotkey('ctrl','v')
pg.sleep(0.5)
pg.press('enter')
pg.sleep(15)

win = pg.getWindowsWithTitle('사진')[0]
win.activate()

pg.hotkey('ctrl','c')

- 이미지 붙여넣기(비정형 데이터 그래프)

In [ ]:
win = pg.getWindowsWithTitle('메일쓰기')[0]
win.activate()

pg.press('enter', presses=2)
pg.hotkey('ctrl','v')
pg.sleep(1)
pg.press('tab', presses=5, interval=0.2)
pg.write('800') #이미지 크기 설정
pg.press('tab', presses=5, interval=0.2)
pg.press('enter')

- 본문 작성 완료

In [ ]:
driver.switch_to.default_content()

- 보내기

In [ ]:
driver.find_element(By.XPATH,'//*[@id="memo_content"]/div[1]/ul[1]/li[1]/a').click()
pg.sleep(5)
try:
    alert = driver.switch_to.alert
    alert.accept()
except: print('계속 진행합니다.')
pg.sleep(3)

---------------

In [ ]:
url = 'https://www.poswel.co.kr'
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
text_xpath = '//*[@id="ctt"]'
text = driver.find_element(By.XPATH, text_xpath).text
text

In [ ]:
len(text)

In [ ]:
a = '포스웰 임직원 여러분!\n잘 알고 있는 바와 같이 윤리경영은 기업의 이익을 가져다 주는 경쟁력의 원천이며 기업 이미지 제고의 수단이기도 합니다. \n이 어려운 시기에 우리 포스웰 구성원 각자가 투철한 윤리의식을 바탕으로 투명하고 올바른 처신을 하는 것이 회사의\n장기적인 성장, 발전에 매우 중요하다는 점을 강조하고 싶습니다.\n그 동안 우리는 디지털 시대에 걸맞은 새로운 시스템 구축을 위해 ERP 시스템을 도입하고 투명경영을 실현하기 위한 제도개혁을\n추진하여 왔으며, 구성원 모두가 업무수행 및 일상생활에서 지키고 따라야 할 윤리규범을 2003년 8월26일 선포하였습니다.\n그러나, 윤리경영의 실현은 시스템이나 제도를 갖추고 윤리규범을 제정하였다고 해서 저절로  이루어지는 것이 아니라\n우리 모두가 최선을 다해 실천을 할 때 비로소 가능할 것입니다.\n 이제 기업의 윤리경영은 우리가 회피하고 유예할 수 있는 대상이 아니라 적극적이고 능동적으로 실천해야 할 과제입니다.\n따라서, 포스웰 임직원으로서 반드시 지켜야 할 윤리규범을 실천하여 윤리규범이 형식에 그치지 않고 내실 있게 실천될 수 있도록\n여러분의 적극적인 협조를 당부 드립니다.\n 아울러, 윤리와 이익이 상충될 때 이익보다는 윤리를 택하는 것이 포스웰의 경영철학임을 명심하고 윤리를 항상 모든 판단과 행동의\n최우선 기준으로 삼아야 하겠습니다.\n 금품수수, 횡령, 성희롱, 정보조작의 4대 비윤리행위에 대해서는 무관용의 원칙을 철저히 지켜나갈것입니다.\n 임직원 여러분! 윤리적 기업문화를 확고히 정착시켜 고객이 가장 거래하고 싶은 기업, 투명하고 신뢰할 수 있는 기업,\n직원들이 행복하게 일하는 일터를 조성함으로써 모든 이해관계자와 동반성장하고 발전하는 포스웰을 만들어 갑시다.\n'

In [ ]:
len(a)

In [ ]:
url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
from bs4 import BeautifulSoup as bs4
soup = bs4(driver.page_source, 'html.parser')

In [ ]:
soup

In [ ]:
list = '#_rankingList0 .list_text_inner > a'
text = soup.select(list)
text

In [ ]:
title = []
for t in text:
    title.append(t.text)
print(title)

In [ ]:
title = []
for i, t in enumerate(text):
    if i%2 == 0: title.append(t.text)
print(title)

In [ ]:
list = "#_rankingList0 .list_tit"
text = soup.select(list)
text

In [ ]:
title = []
for t in text:
    title.append(t.text)
print(title)

In [ ]:
url = 'https://www.data.go.kr/data/15016241/fileData.do'
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
soup = bs4(driver.page_source, 'html.parser')
soup

In [ ]:
table_select = '.fileDataDetail.table'
table_html = soup.select(table_select)
table_html

In [ ]:
type(table_html)

In [ ]:
import pandas

table_list = pd.read_html(str(table_html))
table_list

In [ ]:
type(table_list)

In [ ]:
df = table_list[0]
df

In [ ]:
df = df.rename(columns=df.iloc[0])
df

In [ ]:
df.drop([0], axis=0, inplace=True)

In [ ]:
df